In [2]:
# Here are some imports that are used along this notebook
import math
import itertools
import pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
from time import time
import tensorflow as tf
from collections import OrderedDict
from sklearn.model_selection import KFold
%matplotlib inline
gt0 = time()

/home/hackerx/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
train20_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTrain+_20Percent.txt"
train_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTrain+.txt"
test_nsl_kdd_dataset_path = "NSL_KDD_Dataset/KDDTest+.txt"

In [4]:
col_names = np.array(["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","labels"])

In [5]:
nominal_inx = [1, 2, 3]
binary_inx = [6, 11, 13, 14, 20, 21]
numeric_inx = list(set(range(41)).difference(nominal_inx).difference(binary_inx))

In [6]:
nominal_cols = col_names[nominal_inx].tolist()
binary_cols = col_names[binary_inx].tolist()
numeric_cols = col_names[numeric_inx].tolist()

In [7]:
# Dictionary that contains mapping of various attacks to the four main categories
attack_dict_five_class = {
    'normal': 'normal',
    
    'back': 'DoS',
    'land': 'DoS',
    'neptune': 'DoS',
    'pod': 'DoS',
    'smurf': 'DoS',
    'teardrop': 'DoS',
    'mailbomb': 'DoS',
    'apache2': 'DoS',
    'processtable': 'DoS',
    'udpstorm': 'DoS',
    
    'ipsweep': 'Probe',
    'nmap': 'Probe',
    'portsweep': 'Probe',
    'satan': 'Probe',
    'mscan': 'Probe',
    'saint': 'Probe',

    'ftp_write': 'R2L',
    'guess_passwd': 'R2L',
    'imap': 'R2L',
    'multihop': 'R2L',
    'phf': 'R2L',
    'spy': 'R2L',
    'warezclient': 'R2L',
    'warezmaster': 'R2L',
    'sendmail': 'R2L',
    'named': 'R2L',
    'snmpgetattack': 'R2L',
    'snmpguess': 'R2L',
    'xlock': 'R2L',
    'xsnoop': 'R2L',
    'worm': 'R2L',
    
    'buffer_overflow': 'U2R',
    'loadmodule': 'U2R',
    'perl': 'U2R',
    'rootkit': 'U2R',
    'httptunnel': 'U2R',
    'ps': 'U2R',    
    'sqlattack': 'U2R',
    'xterm': 'U2R'
}

attack_two_class = []
for key in attack_dict_five_class.keys():
    if key == 'normal':
        pass
    else:
        attack_two_class.append(key)

In [8]:
#Load test and train data
train_df = pd.read_csv(train_nsl_kdd_dataset_path, names = col_names)
test_df  = pd.read_csv(test_nsl_kdd_dataset_path , names = col_names)
#Remove column who is total zero i.e.  num_outbound_cmds
train_df = train_df.loc[:, (train_df != 0).any(axis=0)]
test_df = test_df.loc[:, (test_df !=0).any(axis=0)]
numeric_cols.remove('num_outbound_cmds')

In [9]:
# check for null values
train_df[train_df.isnull().any(axis=1)]
test_df[test_df.isnull().any(axis=1)]


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels


In [10]:
test_df.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,labels
0,tcp,private,REJ,0,0,0,0,0,0,0,...,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
0,tcp,private,REJ,0,0,0,0,0,0,0,...,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21
2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,...,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21
0,icmp,eco_i,SF,20,0,0,0,0,0,0,...,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15
1,tcp,telnet,RSTO,0,15,0,0,0,0,0,...,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11


In [11]:
train_labels = train_df.pop('dst_host_srv_rerror_rate')
test_labels = test_df.pop('dst_host_srv_rerror_rate')

In [12]:
train_df = pd.get_dummies(train_df)
test_df = pd.get_dummies(test_df)
train_df.shape

(125973, 121)

In [13]:
train_df.shape

(125973, 121)

In [14]:
train_labels_for_two_class = pd.DataFrame(train_labels.as_matrix(), columns=["class"])
test_labels_for_two_class = pd.DataFrame(test_labels.as_matrix(), columns=["class"])

In [15]:
train_labels_for_two_class.loc[train_labels_for_two_class['class'].isin(attack_two_class) , 'class'] = 1
train_labels_for_two_class.loc[train_labels_for_two_class['class'] == "normal" , 'class'] = 0


#for test labels 
test_labels_for_two_class.loc[test_labels_for_two_class['class'].isin(attack_two_class) , 'class'] = 1
test_labels_for_two_class.loc[test_labels_for_two_class['class'] == "normal" , 'class'] = 0

In [16]:
#min-max mazimazation for the data
from sklearn.preprocessing import MinMaxScaler
train_X = train_df.as_matrix()
train_Y = train_labels_for_two_class.as_matrix()

test_X = train_df.as_matrix()
test_Y = test_labels_for_two_class.as_matrix()
scaler = MinMaxScaler() 
scaler.fit_transform(train_X)
scaler.fit_transform(train_Y)
scaler.fit_transform(test_X)
scaler.fit_transform(test_Y)

/home/hackerx/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


array([[1.],
       [1.],
       [0.],
       ...,
       [1.],
       [0.],
       [1.]])

In [17]:
#Training our auto encoder

import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers



Using TensorFlow backend.


In [18]:
input_dim = train_X.shape[1]
input_layer = Input(shape=(input_dim, ))
encoder = Dense(30, activation="tanh", 
                activity_regularizer=regularizers.l1(10e-5))(input_layer)
decoder = Dense(input_dim, activation='relu')(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)


In [22]:
nb_epoch = 100
batch_size = 400
autoencoder.compile(optimizer='adam', 
                    loss='mean_squared_error', 
                    metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath="model.h5",
                               verbose=0,
                               save_best_only=True)

tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=True)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

history = autoencoder.fit(train_X,train_X ,
                    epochs=nb_epoch,
                    verbose = 1,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_X, test_X),
                    callbacks=[checkpointer, tensorboard]).history

Train on 125973 samples, validate on 125973 samples
Epoch 1/100
125973/125973 [==============================] - 2s 13us/step - loss: 418458185797.0934 - acc: 0.6292 - val_loss: 418458182244.4122 - val_acc: 0.6366
Epoch 2/100
125973/125973 [==============================] - 2s 13us/step - loss: 418458198837.4225 - acc: 0.6305 - val_loss: 418458181981.6029 - val_acc: 0.6324
Epoch 3/100
125973/125973 [==============================] - 2s 14us/step - loss: 418458197685.8115 - acc: 0.6314 - val_loss: 418458181722.7719 - val_acc: 0.6327
Epoch 4/100
125973/125973 [==============================] - 2s 13us/step - loss: 418458177671.8883 - acc: 0.6320 - val_loss: 418458181326.7402 - val_acc: 0.6369
Epoch 5/100
125973/125973 [==============================] - 2s 13us/step - loss: 418458185980.2883 - acc: 0.6323 - val_loss: 418458181078.3527 - val_acc: 0.6249
Epoch 6/100
125973/125973 [==============================] - 2s 13us/step - loss: 418458175224.9964 - acc: 0.6336 - val_loss: 418458167509

Epoch 51/100
125973/125973 [==============================] - 2s 13us/step - loss: 418458089975.3814 - acc: 0.6423 - val_loss: 418458102953.3557 - val_acc: 0.6492
Epoch 52/100
125973/125973 [==============================] - 2s 12us/step - loss: 418458112691.2245 - acc: 0.6431 - val_loss: 418458102635.6944 - val_acc: 0.6408
Epoch 53/100
125973/125973 [==============================] - 2s 12us/step - loss: 418458090297.4214 - acc: 0.6420 - val_loss: 418458102364.4758 - val_acc: 0.6453
Epoch 54/100
125973/125973 [==============================] - 2s 12us/step - loss: 418458104892.7871 - acc: 0.6449 - val_loss: 418458102078.1937 - val_acc: 0.6406
Epoch 55/100
125973/125973 [==============================] - 2s 12us/step - loss: 418458114284.5616 - acc: 0.6460 - val_loss: 418458101225.9477 - val_acc: 0.6491
Epoch 56/100
125973/125973 [==============================] - 1s 11us/step - loss: 418458113295.8432 - acc: 0.6483 - val_loss: 418458100932.7638 - val_acc: 0.6519
Epoch 57/100
125973/12